In [ ]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
from transformers import ViTImageProcessor, ViTForImageClassification, Trainer, TrainingArguments
import evaluate
from PIL import Image

In [ ]:
data_path = '/content/drive/MyDrive/Dataset/image'

dataset = ImageFolder(root=data_path)

print(len(dataset))

1099


In [ ]:
# setting up the dataset
data_path = '/content/drive/MyDrive/Dataset/image'
dataset = ImageFolder(root=data_path)

train_size = int(0.8 * len(dataset))
validation_size = len(dataset) - train_size

training_data, validation_data = random_split(dataset, [train_size, validation_size])

train_loader = DataLoader(training_data, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=32, shuffle=False)

print(f'Total dataset size: {len(dataset)}')
print(f'Training dataset size: {len(training_data)}')
print(f'Validation dataset size: {len(validation_data)}')

Total dataset size: 1099
Training dataset size: 879
Validation dataset size: 220


In [ ]:
model_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_path)
print(processor)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

ViTImageProcessor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}



In [ ]:
def transform(batch):
    inputs = processor(batch['image'], return_tensors="pt")
    inputs['labels'] = batch['labels']
    return inputs

In [ ]:
def collate_fn(batch):
    # preparing batched for dataloader
    pixel_values = torch.stack([transform({'image': image[0], 'labels': image[1]})['pixel_values'].squeeze(0) for image in batch])
    labels = torch.tensor([image[1] for image in batch])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
train_loader = DataLoader(training_data, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(validation_data, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [ ]:
labels = dataset.classes
num_labels = len(labels)

In [ ]:
model = ViTForImageClassification.from_pretrained(
    model_path,
    num_labels=num_labels,
    id2label={str(i): label for i, label in enumerate(labels)},
    label2id={label: str(i) for i, label in enumerate(labels)}
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./vit-local-dataset",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=4,
    fp16=True,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to='none',
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_data,
    eval_dataset=validation_data,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()
metrics = trainer.evaluate()
print(metrics)

Step,Training Loss,Validation Loss,Accuracy
100,0.543400,0.506153,0.818182
200,0.181500,0.387554,0.840909


{'eval_loss': 0.38755369186401367, 'eval_accuracy': 0.8409090909090909, 'eval_runtime': 137.0638, 'eval_samples_per_second': 1.605, 'eval_steps_per_second': 0.102, 'epoch': 4.0}


In [ ]:
!zip -r vit-local-dataset.zip vit-local-dataset/
from google.colab import files
files.download("vit-local-dataset.zip")

updating: vit-local-dataset/ (stored 0%)
updating: vit-local-dataset/checkpoint-220/ (stored 0%)
updating: vit-local-dataset/checkpoint-220/config.json (deflated 51%)
updating: vit-local-dataset/checkpoint-220/rng_state.pth (deflated 24%)
updating: vit-local-dataset/checkpoint-220/optimizer.pt (deflated 7%)
updating: vit-local-dataset/checkpoint-220/training_args.bin (deflated 52%)
updating: vit-local-dataset/checkpoint-220/scheduler.pt (deflated 56%)
updating: vit-local-dataset/checkpoint-220/trainer_state.json (deflated 76%)
updating: vit-local-dataset/checkpoint-220/model.safetensors (deflated 7%)
updating: vit-local-dataset/checkpoint-200/ (stored 0%)
updating: vit-local-dataset/checkpoint-200/config.json (deflated 51%)
updating: vit-local-dataset/checkpoint-200/rng_state.pth (deflated 24%)
updating: vit-local-dataset/checkpoint-200/optimizer.pt (deflated 7%)
updating: vit-local-dataset/checkpoint-200/training_args.bin (deflated 52%)
updating: vit-local-dataset/checkpoint-200/sched

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp -r vit-local-dataset/ /content/drive/MyDrive/